In [ ]:
# Preprocesamiento de Datos para Perceptrón Multicapa
# ========================================
# Este notebook se encarga de:
# 1. Cargar los datos limpios (no normalizados)
# 2. Dividir el conjunto de datos en:
#    - Conjunto de entrenamiento (60%)
#    - Conjunto de validación (20%)
#    - Conjunto de prueba (20%)
# 3. Normalizar los datos usando estadísticas SOLO del conjunto de entrenamiento
# 4. Guardar los conjuntos normalizados y los parámetros de normalización

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Configura el estilo de visualización
plt.style.use('ggplot')
%matplotlib inline

# Añadir el directorio raíz al path para poder importar módulos personalizados
sys.path.append('..')
from utils.config import FEATURE_NAMES

# Crear las carpetas de output si no existen
for folder in ['../output/models', '../output/figures']:
    os.makedirs(folder, exist_ok=True)

# 1. Carga de datos limpios (no normalizados)
# ----------------------------------
print("Cargando datos limpios (no normalizados)...")
df = pd.read_csv('../data/processed/cleaned_data.csv')
print(f"Dimensiones del dataset: {df.shape}")
print(f"Primeras 5 filas del dataset:")
df.head()

# 2. Análisis rápido de los datos cargados
# ----------------------------------------
print("\nInformación del dataset:")
df.info()

print("\nEstadísticas descriptivas:")
df.describe().T

# Verificar que no hay valores nulos
print("\nVerificando valores nulos:")
print(df.isnull().sum().sum())

# Mostrar la distribución de clases
print("\nDistribución de clases (diagnóstico):")
class_counts = df['diagnosis'].value_counts()
print(class_counts)

plt.figure(figsize=(6, 4))
sns.countplot(x='diagnosis', data=df, palette='Set1')
plt.title('Distribución de Diagnósticos')
plt.xlabel('Diagnóstico (0=Benigno, 1=Maligno)')
plt.ylabel('Cantidad')
plt.savefig('../output/figures/class_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# 3. Dividir los datos en conjuntos de entrenamiento, validación y prueba
# ----------------------------------------------------------------------
print("\nDividiendo los datos en conjuntos de entrenamiento, validación y prueba...")

# Separar características (X) y etiquetas (y)
X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

# Primera división: 80% para entrenamiento+validación, 20% para prueba
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Segunda división: 75% del conjunto temporal para entrenamiento (60% del total), 
# 25% para validación (20% del total)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

print(f"Tamaño conjunto de entrenamiento: {X_train.shape[0]} muestras ({X_train.shape[0]/df.shape[0]:.1%} del total)")
print(f"Tamaño conjunto de validación: {X_val.shape[0]} muestras ({X_val.shape[0]/df.shape[0]:.1%} del total)")
print(f"Tamaño conjunto de prueba: {X_test.shape[0]} muestras ({X_test.shape[0]/df.shape[0]:.1%} del total)")

# Verificar la distribución de clases en cada conjunto
print("\nDistribución de clases en cada conjunto:")
print(f"Entrenamiento: {y_train.value_counts().to_dict()}")
print(f"Validación: {y_val.value_counts().to_dict()}")
print(f"Prueba: {y_test.value_counts().to_dict()}")

# Gráfico de distribución de clases por conjunto
plt.figure(figsize=(10, 6))
class_counts = pd.DataFrame({
    'Entrenamiento': y_train.value_counts(),
    'Validación': y_val.value_counts(),
    'Prueba': y_test.value_counts()
})
class_counts.plot(kind='bar')
plt.title('Distribución de Clases por Conjunto')
plt.xlabel('Diagnóstico (0=Benigno, 1=Maligno)')
plt.ylabel('Cantidad')
plt.tight_layout()
plt.savefig('../output/figures/class_distribution_by_set.png', dpi=300, bbox_inches='tight')
plt.show()

# 4. Normalización de datos (Z-score)
# -----------------------------------
print("\nNormalizando datos con Z-score usando SOLO estadísticas del conjunto de entrenamiento...")

# Inicializar el normalizador
scaler = StandardScaler()

# Ajustar el normalizador SOLO a los datos de entrenamiento
scaler.fit(X_train)

# Transformar todos los conjuntos usando las estadísticas del conjunto de entrenamiento
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Crear DataFrames con los datos normalizados para análisis
X_train_norm = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_val_norm = pd.DataFrame(X_val_scaled, columns=X_val.columns)
X_test_norm = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Guardar medias y desviaciones estándar para uso futuro
normalization_params = {
    'means': {col: float(scaler.mean_[i]) for i, col in enumerate(X_train.columns)},
    'stds': {col: float(scaler.scale_[i]) for i, col in enumerate(X_train.columns)}
}

with open('../output/normalization_params.json', 'w') as f:
    json.dump(normalization_params, f, indent=4)
print(f"Guardados parámetros de normalización en ../output/normalization_params.json")

# Visualizar efecto de la normalización en algunas características
print("\nEfecto de la normalización (datos de entrenamiento):")
fig, axs = plt.subplots(2, 2, figsize=(14, 10))
axs = axs.flatten()

selected_features = ['f01', 'f02', 'f07', 'f08']  # Importantes según análisis previo

for i, feature in enumerate(selected_features):
    # Datos originales
    sns.histplot(X_train[feature], color='blue', alpha=0.5, 
                 label='Original', ax=axs[i], kde=True)
    
    # Datos normalizados
    sns.histplot(X_train_norm[feature], color='red', alpha=0.5, 
                 label='Normalizado', ax=axs[i], kde=True)
    
    axs[i].set_title(f'{feature} ({FEATURE_NAMES[feature]})')
    axs[i].axvline(x=0, color='black', linestyle='--', alpha=0.7)
    axs[i].legend()

plt.tight_layout()
plt.savefig('../output/figures/normalization_effect.png', dpi=300, bbox_inches='tight')
plt.show()

# 5. Preparar los datos para el modelo
# -----------------------------------
# Para un perceptrón multicapa sin usar frameworks como TensorFlow o PyTorch,
# convertimos los datos a arrays NumPy

# Conversión a arrays NumPy
X_train_array = X_train_scaled
X_val_array = X_val_scaled
X_test_array = X_test_scaled

y_train_array = y_train.to_numpy().reshape(-1, 1)  # Reshape para tener la dimensión correcta
y_val_array = y_val.to_numpy().reshape(-1, 1)
y_test_array = y_test.to_numpy().reshape(-1, 1)

print(f"\nForma de los arrays de características:")
print(f"X_train: {X_train_array.shape}")
print(f"X_val: {X_val_array.shape}")
print(f"X_test: {X_test_array.shape}")

print(f"\nForma de los arrays de etiquetas:")
print(f"y_train: {y_train_array.shape}")
print(f"y_val: {y_val_array.shape}")
print(f"y_test: {y_test_array.shape}")

# 6. Guardar los conjuntos de datos procesados
# -------------------------------------------
# Guardar como CSV para uso posterior
print("\nGuardando conjuntos de datos procesados...")

# Guardar conjuntos de entrenamiento, validación y prueba normalizados
datasets_norm = {
    'train': (X_train_norm, y_train),
    'val': (X_val_norm, y_val),
    'test': (X_test_norm, y_test)
}

for name, (X_set, y_set) in datasets_norm.items():
    # Crear un DataFrame con características y etiqueta
    combined_df = X_set.copy()
    combined_df['diagnosis'] = y_set.values
    
    # Guardar en CSV
    filepath = f'../data/processed/{name}_set_normalized.csv'
    combined_df.to_csv(filepath, index=False)
    print(f"Guardado conjunto normalizado {name} en {filepath}")

# Guardar los parámetros de división para reproducibilidad
split_params = {
    'train_size': X_train.shape[0],
    'val_size': X_val.shape[0],
    'test_size': X_test.shape[0],
    'train_pct': float(X_train.shape[0]/df.shape[0]),
    'val_pct': float(X_val.shape[0]/df.shape[0]),
    'test_pct': float(X_test.shape[0]/df.shape[0]),
    'stratify': True,
    'random_state': 42
}

with open('../output/split_params.json', 'w') as f:
    json.dump(split_params, f, indent=4)
print(f"Guardados parámetros de división en ../output/split_params.json")

# 7. Guardar arrays NumPy para uso directo en el modelo
# Estos archivos son más eficientes para cargar directamente en el modelo
print("\nGuardando arrays NumPy para uso directo en el modelo...")
np.save('../data/processed/X_train.npy', X_train_array)
np.save('../data/processed/y_train.npy', y_train_array)
np.save('../data/processed/X_val.npy', X_val_array)
np.save('../data/processed/y_val.npy', y_val_array)
np.save('../data/processed/X_test.npy', X_test_array)
np.save('../data/processed/y_test.npy', y_test_array)
print("Arrays NumPy guardados correctamente.")

# 8. Verificación de la distribución de características normalizadas
# ---------------------------------------------------
print("\nVerificando la distribución de características normalizadas entre conjuntos...")

# Seleccionar algunas características importantes basadas en análisis previo
important_features = ['f01', 'f08', 'f24', 'f28']

fig, axs = plt.subplots(2, 2, figsize=(14, 10))
axs = axs.flatten()

for i, feature in enumerate(important_features):
    sns.kdeplot(X_train_norm[feature], ax=axs[i], label='Entrenamiento', color='blue')
    sns.kdeplot(X_val_norm[feature], ax=axs[i], label='Validación', color='green')
    sns.kdeplot(X_test_norm[feature], ax=axs[i], label='Prueba', color='red')
    
    axs[i].set_title(f'Distribución de {feature} normalizado ({FEATURE_NAMES[feature]})')
    axs[i].set_xlabel('Valor normalizado')
    axs[i].set_ylabel('Densidad')
    axs[i].axvline(x=0, color='black', linestyle='--', alpha=0.3)
    axs[i].legend()

plt.tight_layout()
plt.savefig('../output/figures/feature_distribution_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# 9. Resumen del preprocesamiento
# -----------------------------
print("\n=== Resumen del Preprocesamiento ===")
print(f"Dataset original: {df.shape[0]} muestras, {df.shape[1]} características")
print("\nDivisión de datos:")
print(f"- Entrenamiento: {X_train.shape[0]} muestras ({X_train.shape[0]/df.shape[0]:.1%})")
print(f"- Validación: {X_val.shape[0]} muestras ({X_val.shape[0]/df.shape[0]:.1%})")
print(f"- Prueba: {X_test.shape[0]} muestras ({X_test.shape[0]/df.shape[0]:.1%})")

print("\nNormalización:")
print("- Método: Z-score (StandardScaler)")
print("- Estadísticas calculadas SOLO con datos de entrenamiento")
print("- Mismo scaler aplicado a todos los conjuntos")

print("\nArchivos generados:")
print("- CSVs con conjuntos normalizados: train_set_normalized.csv, val_set_normalized.csv, test_set_normalized.csv")
print("- Arrays NumPy: X_train.npy, y_train.npy, X_val.npy, y_val.npy, X_test.npy, y_test.npy")
print("- Parámetros de normalización: normalization_params.json")
print("- Parámetros de división: split_params.json")

print("\nPróximos pasos:")
print("1. Desarrollar la arquitectura del perceptrón multicapa")
print("2. Implementar el algoritmo de entrenamiento")
print("3. Entrenar el modelo con los datos de entrenamiento")
print("4. Validar el modelo con los datos de validación")
print("5. Evaluar el rendimiento final con los datos de prueba")